# Test comparison across models

In this notebook, we compare all the developed models with the company’s currently deployed models on a held-out block of data (cold semester 2024–2025).

The company’s models are based on classical machine-learning approaches:

* [**XGBoost**](https://arxiv.org/abs/1603.02754): an optimized gradient-boosting framework that builds an ensemble of regression trees sequentially, where each new tree corrects the errors of the previous ones. It includes regularization, efficient handling of missing values, and parallelized computation for high predictive performance and scalability.

* **[SVMR](https://arxiv.org/abs/1603.02754) (Support Vector Machine Regression)**: a kernel-based regression model that finds the function best fitting the data by maximizing the margin around the regression hyperplane, offering strong performance on nonlinear relationships through the use of kernel transformations. 

### Final Model Testing Procedure

Final model evaluation is performed via **rolling cross-validation** over **one year of data** (May 2024 → May 2025). Evaluation origins advance **daily**, and at each origin we generate forecasts for either the **daily horizon** (24 hours) or the **weekly horizon** (168 hours), providing dense temporal coverage. To control computation, the model is **refit only once every seven days**; intermediate origins reuse the most recent fit.

In practice, for each day between mid-May 2024 and mid-May 2025:

1. **Refit schedule:** If it’s the first origin (test window) or the **7th, 14th, 21st, …** (i.e., `origin_index % 7 == 0`), refit on all data available up to that day; otherwise, reuse the last fitted model.
2. **Forecast:** From that origin, produce an **h-step** forecast (h = 24 for the daily horizon; h = 168 for the weekly horizon).

Forecasts are stored and subsequently evaluated and compared across models in the `Test comparison.ipynb` notebook.

### Evaluation Summary


The evaluation results reveal the following key insights:

1. **SARIMAX performs comparably to the company’s existing machine learning models**, indicating that a well-calibrated statistical model can match the performance of more complex, ML-based approaches.

2. **MSTL performs similarly to the naïve baseline across most time series**, with the notable exception of **series F5**, where it comes close to SARIMAX. This can be attributed to F5's pronounced **weekly seasonality**, which MSTL captures effectively, unlike the naïve model. This suggests that **MSTL is particularly well-suited as a baseline model for time series with strong weekly patterns**.

## Import Libreries and Data

You can run the notebook in two ways:

1. **Google Colab**: place the project folder `heat-forecast` in **MyDrive**. The setup cell below will mount Drive and automatically add `MyDrive/heat-forecast/src` to `sys.path` so `import heat_forecast` works out of the box.

2. **Local machine**:

   * **Installing our package:** from the project root, run `pip install -e .` once (editable install). Then you can open the notebook anywhere and import the package normally.
   * **Alternative:** if you’re running the notebook from `.../heat-forecast/notebooks/` without installing the package, the setup cell will detect `../src` and automatically add it to `sys.path`.

In [ ]:
# === Do not edit below ===
# --- Detect if running on Google Colab & Set base dir ---
# %cd /home/giovanni.lombardi/heat-forecast/notebooks
import subprocess
from pathlib import Path
import sys

def in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

# Install required packages only if not already installed
def pip_install(pkg: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Set base directory and handle environment
if in_colab():
    # Make sure IPython is modern (avoids the old %autoreload/imp issue if you ever use it)
    pip_install("ipython>=8.25")
    pip_install("ipykernel>=6.29")
    
    def install(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    for pkg in ["statsmodels", "statsforecast", "mlforecast"]:
        pip_install(pkg)

    # Mount Google Drive
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')

    # Set base directory to your Drive project folder
    BASE_DIR = Path('/content/drive/MyDrive/heat-forecast')

    # Add `src/` to sys.path for custom package imports
    SRC_PATH = BASE_DIR / 'src'
    if str(SRC_PATH) not in sys.path:
        sys.path.append(str(SRC_PATH))

    # Sanity checks (helpful error messages if path is wrong)
    assert SRC_PATH.exists(), f"Expected '{SRC_PATH}' to exist. Fix BASE_DIR."
    pkg_dir = SRC_PATH / "heat_forecast"
    assert pkg_dir.exists(), f"Expected '{pkg_dir}' package directory."
    init_file = pkg_dir / "__init__.py"
    assert init_file.exists(), f"Missing '{init_file}'. Add it so Python treats this as a package."

else:
    # Local: either rely on editable install (pip install -e .) or add src/ when running from repo
    # Assume notebook lives in PROJECT_ROOT/notebooks/
    BASE_DIR = Path.cwd().resolve().parent
    SRC_PATH = BASE_DIR / "src"

    added_src = False
    if (SRC_PATH / "heat_forecast").exists() and str(SRC_PATH) not in sys.path:
        sys.path.append(str(SRC_PATH))
        added_src = True

# --- Logging setup ---
import logging
from zoneinfo import ZoneInfo
from datetime import datetime

LOG_DIR  = (BASE_DIR / "logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / "run.log"
PREV_LOG = LOG_DIR / "run.prev.log"

# If there's a previous run.log with content, archive it to run.prev.log
if LOG_FILE.exists() and LOG_FILE.stat().st_size > 0:
    try:
        # Replace old run.prev.log if present
        if PREV_LOG.exists():
            PREV_LOG.unlink()
        LOG_FILE.rename(PREV_LOG)
    except Exception as e:
        # Fall back to truncating if rename fails (e.g., file locked)
        print(f"[warn] Could not archive previous log: {e}. Truncating current run.log.")
        LOG_FILE.write_text("")

# Configure logging: fresh file for this run + echo to notebook/stdout
file_handler   = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
stream_handler = logging.StreamHandler(sys.stdout)

fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(name)s | %(message)s",
                        datefmt="%m-%d %H:%M:%S")
file_handler.setFormatter(fmt)
stream_handler.setFormatter(fmt)

root = logging.getLogger()
root.handlers[:] = [file_handler, stream_handler]  # replace handlers (important in notebooks)
root.setLevel(logging.INFO)

# Use Rome time
logging.Formatter.converter = lambda *args: datetime.now(ZoneInfo("Europe/Rome")).timetuple()

logging.captureWarnings(True)
logging.info("=== Logging started (fresh current run) ===")
logging.info("Previous run (if any): %s", PREV_LOG if PREV_LOG.exists() else "none")

if added_src:
    logging.info("heat_forecast not installed; added src/ to sys.path")
else:
    logging.info("heat_forecast imported without modifying sys.path (likely installed)")

OPTUNA_DIR = BASE_DIR / "results" / "finetuning" / "lstm"
OPTUNA_DIR.mkdir(parents=True, exist_ok=True)
logging.info("BASE_DIR (make sure it's '*/heat-forecast/', else cd and re-run): %s", BASE_DIR)
logging.info("LOG_DIR: %s", LOG_DIR)
logging.info("OPTUNA_DIR: %s", OPTUNA_DIR)

Ensure [compatibility with Numba](https://numba.readthedocs.io/en/stable/user/installing.html#numba-support-info).

In [ ]:
# === Do not edit below ===
import sys, numpy, numba
logging.info("=== Current Environment ===")
logging.info("Python : %s", sys.version.split()[0])
logging.info("NumPy  : %s", numpy.__version__)
logging.info("Numba  : %s", numba.__version__)

Imports:

In [ ]:
# --- IPython Magic ---
%load_ext autoreload
%autoreload 2

# --- Standard Library ---
import sys
import yaml
import warnings
import logging
from collections import defaultdict
from datetime import datetime
from tqdm.notebook import tqdm
from typing import Any, Dict, Optional, Tuple, Iterable, Callable
from functools import partial

# --- Logging Configuration ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%H:%M:%S",
    force=True,
    stream=sys.stdout
)

# --- Data Handling ---
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)

# --- Plotting & Visualization ---
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML

# Set plot style and parameters
plt.style.use("seaborn-v0_8")
mpl.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 14,
    'axes.grid': True,
    'axes.grid.which': 'both',
})

# --- Forecasting Models ---
from statsforecast import StatsForecast
from statsforecast.models import SeasonalNaive

# --- Project-Specific Utilities ---
from heat_forecast.utils.plotting import (
    plotly_cutoffs_with_exog, display_scrollable,
    add_season_background, plotly_daily_seasonality,
    plot_acf_diagnostics
)

from heat_forecast.utils.evaluation import (
    evaluate_cv_forecasts, display_cv_summary, cv_evaluation_summary,
    compute_loss_diffs, plotly_cv_metric_by_cutoff,
    by_horizon_preds, compute_error_stats_by_horizon,
    plotly_models_vs_horizon
)
from heat_forecast.utils.cv_utils import display_info_cv

logging.info("All imports successful.")

Import pre-elaborated data.

In [ ]:
heat_path = BASE_DIR / 'data' / 'timeseries_preprocessed' / 'heat.csv'
aux_path = BASE_DIR / 'data' / 'timeseries_preprocessed' / 'auxiliary.csv'
heat_df = pd.read_csv(heat_path, parse_dates=['ds'])
aux_df = pd.read_csv(aux_path, parse_dates=['ds'])

## Load Saved Test Results

In [ ]:
TESTS_DIR = BASE_DIR / 'results' / 'test'
cv_groups = defaultdict(list)    # {(Fi, horizon): [(cv_df, run_dir)]}
run_tracker = defaultdict(list)  # {(model, Fi, horizon): [(timestamp, run_dir)]}
model_folder_to_name_map = {
    "mstl": "MSTL",
    "sarimax": "SARIMAX",
    "lstm": "LSTM",
    "xgboost": "XGBoost",
    "svmr": "SVMR",
}
cutoffs = None  # Will be set to the unique cutoffs from the first loaded DataFrame

# --- Collect all valid run_dirs ---
for model_dir in TESTS_DIR.iterdir():
    if not model_dir.is_dir():
        continue

    for run_dir in model_dir.iterdir():
        if not run_dir.is_dir():
            logging.debug(f"Unexpected non-directory in folder: {run_dir.relative_to(TESTS_DIR)}. "
                            f"Non-directory: {run_dir}.")
            continue

        parts = run_dir.name.split('_')
        if len(parts) < 3 or not parts[0].startswith('F'):
            logging.debug(f"Skipping folder {run_dir.relative_to(TESTS_DIR)} due to unexpected format. "
                            f"Folder name: {run_dir.name}. "
                            f"Expected format: <unique_id>_<horizon_type>_test_<model_name>_<timestamp>.")
            continue

        fi = parts[0]
        horizon = parts[1]
        timestamp_str = parts[-1]
        model_run = parts[-2]
        if model_run not in model_folder_to_name_map.keys():
            if model_run.split('-')[0] in model_folder_to_name_map.keys(): # -> it's a variant of that model, ignore
                continue
            logging.debug(f"Model '{model_run}' (from run {run_dir.relative_to(TESTS_DIR)}) not recognized. "
                            f"Update the dict model_folder_to_name_map at the top of this cell."
                            f"Using model_run as model name.")
        if model_dir.name not in model_folder_to_name_map.keys():
            logging.debug(f"Model for folder '{model_dir.name}' not recognized. "
                            f"Update the dict model_folder_to_name_map at the top of this cell."
                            f"Using folder name as model name.")
        model = model_folder_to_name_map.get(model_dir.name, model_dir.name)

        try:
            timestamp = datetime.strptime(timestamp_str, "%Y%m%dT%H%M%S")
        except ValueError:
            logging.debug(f"Skipping run with invalid timestamp: {run_dir}")
            continue

        run_tracker[(model, fi, horizon)].append((timestamp, run_dir))

# ----------- Select latest runs and load them -----------
for key, runs in run_tracker.items():
    model, fi, horizon = key
    runs_sorted = sorted(runs, key=lambda x: x[0], reverse=True)

    if len(runs_sorted) > 1:
        logging.debug(f"Multiple runs found for model={model}, id={fi}, horizon={horizon}. Using latest only.")

    latest_run_dir = runs_sorted[0][1]
    cv_path = latest_run_dir / "cv_df.parquet"

    if not cv_path.exists():
        logging.debug(f"cv_df.parquet not found for latest run: {latest_run_dir}")
        continue

    try:
        df = pd.read_parquet(cv_path)
    except Exception as e:
        logging.debug(f"Failed to load {cv_path}: {e}")
        raise

    if not {'unique_id', 'ds'}.issubset(df.columns):
        raise ValueError(f"DataFrame from {cv_path} does not contain required columns ['unique_id', 'ds']")

    df.drop(columns=['y'], inplace=True, errors='ignore')  # Drop 'y' column if it exists
    df = df.set_index(['unique_id', 'ds', 'cutoff'])
    df.index.names = ['unique_id', 'ds', 'cutoff']
    cv_groups[(fi, horizon)].append((df, latest_run_dir, model))

# Build tidy records
rows = []
for (fi, horizon), values in cv_groups.items():
    for el in values:
        rows.append({
            "unique_id": fi,
            "horizon": horizon,
            "model": el[2],
            "run": str(el[1].relative_to(TESTS_DIR)),
        })

# Log it nicely
df = (
    pd.DataFrame(rows)
      .sort_values(["unique_id", "horizon", "model", "run"])
      .reset_index(drop=True)
)
df = df.set_index(["unique_id", "horizon", "model"])[["run"]]
with pd.option_context("display.max_rows", None, "display.width", 160):
    logging.info("Successfully selected the following runs:")
    display(df)

# ----------- Merge DataFrames -----------
merged_results_tmp = {} # {(Fi, horizon): (merged_df, for_cv)}
times = {}  # {(Fi, horizon): times_df}

for (fi, horizon), value in cv_groups.items():
    dfs, run_dirs, models = zip(*value)  

    rows = []           

    for i in range(len(dfs)):
        run_dir = run_dirs[i]
        model_name = models[i]
        df = dfs[i]

        # default record (in case metadata is missing)
        rec = {
            "model": model_name,
            "run_dir": str(run_dir),
            "step_size": pd.NA,
            "test_hours": pd.NA,
            "end_test_cv": pd.NA,
            "avg_el_per_fit": pd.NA,
        }

        metadata_path = run_dir / "metadata.yaml"
        try:
            with open(metadata_path, "r") as f:
                md = yaml.safe_load(f)
            if not isinstance(md, dict):
                md = {}
        except FileNotFoundError:
            logging.debug(f"Metadata file not found for {run_dir}.")
            rows.append(rec)
            continue

        for_cv = md.get("for_cv", {}) if isinstance(md.get("for_cv", {}), dict) else {}

        ss  = for_cv.get("step_size", pd.NA)
        th  = for_cv.get("test_hours", pd.NA)
        etc = for_cv.get("end_test_cv", pd.NA)
        ae  = md.get("avg_el_per_fit", pd.NA)
        ss  = pd.to_numeric(ss, errors="coerce") if ss is not pd.NA else pd.NA
        th  = pd.to_numeric(th, errors="coerce") if th is not pd.NA else pd.NA
        etc = pd.to_datetime(etc, errors="coerce") if etc is not pd.NA else pd.NA
        ae  = pd.to_numeric(ae, errors="coerce") if ae is not pd.NA else pd.NA

        if horizon == 'week' and ss is not pd.NA and ss == 24:
            logging.debug(f"Adjusting step_size from 24 to 168 for weekly horizon in {run_dir}.")
            ss = 24*7
            h = 24*7
            n_windows = (th - h) // ss + 1
            all_cutoffs = pd.to_datetime(df.reset_index()['cutoff'].unique())
            weekly_cutoffs = [
                etc - pd.Timedelta(hours=th) + pd.Timedelta(hours=ss * i)
                for i in range(n_windows)
            ]
            expected = all_cutoffs[::7]
            assert set(weekly_cutoffs) == set(expected), (
                f"Weekly cutoffs do not match expected for {run_dir}: \n"
                f"actual = \n{weekly_cutoffs}, \n"
                f"expected = \n{expected}, \n" 
                f"th_actual = {(weekly_cutoffs[-1] + pd.Timedelta(hours=h) - weekly_cutoffs[0]).total_seconds() // 3600} \n"
                f"th_expected = {(expected[-1] + pd.Timedelta(hours=h) - expected[0]).total_seconds() // 3600}"
            )
            etc_actual = weekly_cutoffs[-1] + pd.Timedelta(hours=h)
            th_actual = int((etc_actual - weekly_cutoffs[0]).total_seconds() // 3600)
            etc = etc_actual
            th = th_actual
            mask = df.index.get_level_values('cutoff').isin(weekly_cutoffs)
            df = df[mask]

        rec.update({
            "step_size": ss,
            "test_hours": th,
            "end_test_cv": etc,
            "avg_el_per_fit": ae,
        })

        to_log = []
        if pd.isna(ss) or pd.isna(th) or pd.isna(ae):
            if pd.isna(ss) or pd.isna(th):
                to_log.append("Missing CV params")
            if pd.isna(ae):
                to_log.append("Missing elapsed")
            logging.debug(f"{' & '.join(to_log)} for {run_dir}.")

        rows.append(rec)

        if i == 0:
            merged_df = df.copy()
        else:
            merged_df = merged_df.join(df, how='outer')

    merged_df = merged_df.reset_index()

    # Build the DataFrame you expect: one row per model/run
    for_cv_df = pd.DataFrame.from_records(rows, index='model')
    if len(for_cv_df['step_size'].dropna().unique()) > 1:
        raise ValueError(f"Different step sizes detected for Fi={fi}, horizon={horizon}:\n {for_cv_df['step_size'].dropna().unique()}")
    if len(for_cv_df['test_hours'].dropna().unique()) > 1:
        raise ValueError(f"Different test hours detected for Fi={fi}, horizon={horizon}:\n {for_cv_df['test_hours'].dropna().unique()}")
    if len(for_cv_df['end_test_cv'].dropna().unique()) > 1:
        raise ValueError(f"Different end_test_cv detected for Fi={fi}, horizon={horizon}:\n {for_cv_df['end_test_cv'].dropna().unique()}")

    # we assume at least one metadata was found
    for_cv = {
        'step_size': for_cv_df['step_size'].dropna().unique()[0],
        'test_hours': for_cv_df['test_hours'].dropna().unique()[0],
        'end_test_cv': for_cv_df['end_test_cv'].dropna().unique()[0],
    }

    merged_results_tmp[(fi, horizon)] = (merged_df, for_cv)
    times[(fi, horizon)] = for_cv_df["avg_el_per_fit"]

merged_results = {}  # {(Fi, horizon): merged_df}

# ----------- Add Naive forecast -----------
for (fi, horizon), (df, for_cv) in merged_results_tmp.items():
    naive_model = SeasonalNaive(season_length=24, alias='Naive24h')
    sf = StatsForecast(models=[naive_model], freq='h')

    heat_id_df = heat_df[heat_df['unique_id'] == fi].copy()
    
    h = 24*7 if horizon == 'week' else 24
    step_size = for_cv['step_size']
    test_hours = for_cv['test_hours']
    end_test_cv = for_cv['end_test_cv']
    n_windows = (test_hours - h) // step_size + 1
    full_df = heat_id_df[heat_id_df['ds'] <= end_test_cv]  

    cv_naive = sf.cross_validation(
        h=h,
        df=full_df,
        n_windows=n_windows,
        step_size=step_size,  
        test_size=test_hours,  
        input_size=None,  
        refit=True,  
    )

    df = df.merge(
        cv_naive,
        on=['unique_id', 'ds', 'cutoff'],
        how='outer',
    )

    merged_results[(fi, horizon)] = df.copy()

## Series F1

Plot target during test period:

In [ ]:
def plot_target_series(heat_df: pd.DataFrame, unique_id: str):
    y = heat_df[heat_df['unique_id'] == unique_id].set_index('ds')['y']
    y_test = y[y.index > pd.Timestamp('2024-09-01')]

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=y_test.index,
            y=y_test.values,
            mode="lines",
        )
    )
    fig.update_layout(
        title=f"Hourly target series - {unique_id}",
        xaxis_title="Time",
        yaxis_title="Heat demand (kW/h)",
        hovermode="x unified",
        template="plotly_white",
        width=1000,
        height=500,
    )

    # Add background fill by season
    add_season_background(fig, y_test.index.min(), y_test.index.max())

    html = fig.to_html(include_plotlyjs="inline", full_html=False)
    display(HTML(html))

plot_target_series(heat_df, 'F1')

Plot daily mean of target:

In [ ]:
def plot_target_series_daily_agg(heat_df: pd.DataFrame, unique_id: str):
    y = heat_df[heat_df['unique_id'] == unique_id].set_index('ds')['y']
    y_test = y[y.index > pd.Timestamp('2024-09-01')]
    y_test_daily = y_test.resample('D').mean()

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=y_test_daily.index,
            y=y_test_daily.values,
            mode="lines",
        )
    )
    fig.update_layout(
        title="Daily average of the target series",
        xaxis_title="Time",
        yaxis_title="Average heat demand (kW/h)",
        hovermode="x unified",
        template="plotly_white",
        width=1000,
        height=500,
    )

    # Add background fill by season
    add_season_background(fig, y_test.index.min(), y_test.index.max())

    html = fig.to_html(include_plotlyjs="inline", full_html=False)
    display(HTML(html))

plot_target_series_daily_agg(heat_df, 'F1')

### Daily horizon

In [ ]:
cv_df = merged_results[('F1', 'day')].copy()
times_df = times[('F1', 'day')].copy()
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F1_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F1_day_df)

##### Per-Window Error Statistics

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

The main insights emerging from this initial examination of the error statistics are summarized below.

* **Accuracy.**
  The results suggest that the LSTM model **slightly outperforms** the alternative approaches. However, the **standard deviations are relatively large** compared with the differences in mean and median performance, approximately two to three times higher, even when considering the distributions of loss differences. This pattern indicates that the observed improvements may **not be statistically significant**.

  In contrast, the **win-rate analysis** shows that the LSTM achieves the best performance in **more than twice as many validation windows** as any other model (40% versus ≤17% for the others), providing stronger evidence of a genuine performance advantage. The elevated standard deviations, including those derived from per-window differences, may partly reflect **strong autocorrelations** at seasonal lags (24 h and 168 h) and **heteroscedasticity** in the data, likely induced by **seasonal regime shifts** between summer and winter periods.

  It is also noteworthy that, when restricting the comparison to the two best-performing models, **LSTM and XGBoost**, the LSTM achieves a **win rate of 68%** for MAE/NMAE (and consequently 32% for XGBoost) and **72% for RMSE and SMAPE**, which further supports the conclusion that the LSTM provides a **consistent accuracy improvement** over XGBoost.

* **Computational cost.**
  There is a **substantial difference in computational cost**: less than one second per fit for the classical ML models compared with roughly one minute per fit for the LSTM and SARIMAX models. This gives XGBoost a considerable advantage over SARIMAX. The improvement in accuracy provided by the LSTM therefore comes at the expense of **significantly higher computation time**.

  However, this does not make the LSTM **operationally disadvantageous**. The practical value of the trade-off between accuracy and computation depends heavily on the specific application scenario:

  * First, depending on how the forecasts are used in practice, the required level of accuracy may vary, and even a modest but consistent improvement can be valuable.
  * Second, model fitting typically occurs **once per week per series**, which was also the setup used in our experiments. Therefore, although the relative increase in computational time is large, the **absolute cost remains manageable** given the infrequency of refitting, unless forecasts need to be produced continuously for a large number of series.

**Overall**, while the LSTM entails a significantly higher computational cost, its **superior consistency across validation windows** suggests that it remains a competitive choice when forecast accuracy is a key requirement.

##### Visualization of Per-Window Errors

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX'],
)
display_scrollable(fig) 

The periods during which the LSTM model shows the greatest relative advantage over **SARIMAX** are as follows:

* **15–18 October:** the initial days when heating demand begins to rise;
* **3–13 November:** the phase exhibiting the steepest increase in heat demand, as shown in the daily mean demand plot above.

For **XGBoost**, the loss differences are more uniform, although there are still a few days aroung Novermber 3 for which LSTM consistently ouperforms it. 

These intervals coincide with **transitional phases** between the warm and cold seasons, suggesting that the main difficulty for the SARIMAX lies in capturing regime shifts and nonstationary dynamics associated with rapid, temperature-driven changes in heating behaviour.

For **SARIMAX**, this limitation is conceptually expected. The model explicitly partitions the year into two regimes—warm and cold—based on a rolling average of temperature, with the cold period variance stabilized via a targeted Box–Cox transformation. However, this coarse regime specification tends to produce abrupt behavioural changes between the two states. As a result, when the actual demand transition is gradual or delayed relative to temperature (for example, due to social or operational factors), the model’s response can become temporally misaligned with the true dynamics of heating demand.

In addition, all SARIMAX parameters were estimated via **maximum likelihood** over the entire training horizon. This means that the coefficients of cold-period exogenous variables (the weekly Fourier terms and the temperature-derived regressors defined specifically for that regime) are primarily optimized for the **dominant peak regime** (the winter season) rather than for the shorter transitional intervals. 

This design choice was intentional: the goal was to prioritise accuracy during the high-demand, operationally critical winter period. Approaches that aim to improve performance during the transitions often do so at the expense of accuracy in this core regime. Hence, the observed weakness in transition periods reflects not only the structure of the model but also a deliberate **bias–variance trade-off** favouring the stability and reliability of winter forecasts.

Nonetheless, this also highlights an intrinsic limitation of SARIMAX (and, more generally, of classical statistical models) in handling **heteroscedastic** and **strongly nonstationary** time series. The use of exogenous temperature variables and a Box–Cox transformation mitigates this issue to some extent, but cannot fully capture the nonlinear, evolving nature of the underlying process.

By contrast, machine learning models such as XGBoost and LSTM can inherently model **nonlinear temporal dependencies** and **continuous regime evolution** without the need for explicit segmentation or variance stabilization. Their internal state representations allow them to adapt flexibly to gradual structural shifts in the data-generating process, such as the onset or offset of the heating season.
This capacity to learn smooth and context-dependent transitions explains their superior performance during these critical changeover periods and underscores their strength in modeling **nonstationary, seasonally evolving demand patterns**.

In [ ]:
_ = plot_acf_diagnostics(
    df=autumn_F1_day_df,
    cols_to_plot=['SARIMAX', 'XGBoost', 'LSTM'],
    acf_max_lag=200,
)

##### Visualization of Forecasts

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['MSTL', 'Naive24h'],
    highlight_dayofweek=True,
    #order_of_models=['SARIMAX', 'MSTL'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F1_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
winter_target = heat_df[
    (heat_df['unique_id'] == 'F1') &
    (heat_df['ds'] >= pd.Timestamp('2024-12-21')) &
    (heat_df['ds'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F1_day_df)

##### Per-Window Error Statistics

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,#[['LSTM', 'XGBoost', 'unique_id', 'ds', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
ws = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='Naive24h'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='smape',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=400,
    grayscale_safe=True,
    width=800
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-MSTL', 'LD-SARIMAX'],
    as_lineplot=False,
    grayscale_safe=False,
    width=1100
)
display_scrollable(fig) 

In [ ]:
from dieboldmariano import dm_test

L = all_results[all_results['metric'] == "mae"]['LSTM'].values
X = all_results[all_results['metric'] == "mae"]['XGBoost'].values
T = np.zeros(len(L))

dm_test(T, L, X, h=7)

In [ ]:
from scipy import stats

t_stat, p_val = stats.ttest_rel(L, X)  # equivalent
print(t_stat, p_val)

##### Analysis of per-hour-ahead errors

In the winter period, when forecasting accuracy is most critical due to peak heat demand, the series exhibits lower heteroskedasticity because the regime transitions typical of autumn and spring do not occur. Therefore, we include an additional analysis of forecast errors by hour ahead horizon, which, given that forecasts always start at midnight, can also be interpreted as an analysis by hour of day.

In [ ]:
by_horizon_dict = by_horizon_preds(winter_F1_day_df)
per_h_summ = compute_error_stats_by_horizon(
    by_horizon_dict=by_horizon_dict,
    target_df=heat_df,
    nmae_period=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21')),
    log_every=5,
    show_per_cutoff=True,
    evaluate_fn=evaluate_cv_forecasts,
    summarize_fn=cv_evaluation_summary,
    by_cutoff_plot_fn=partial(
        plotly_cv_metric_by_cutoff, 
        models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR'],
        width=2000
    ),
)


In [ ]:
# Target grouped by month-dayhour
plotly_daily_seasonality(
    winter_target,
    show_legend=True,
    colors=px.colors.qualitative.Dark2
)

# Errors gouped by horizon
agg = 'mean'
plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                         models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR', 'MSTL', 'Naive24h'],
                         metric='mae', 
                         agg=agg)
plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                         models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR', 'MSTL', 'Naive24h'],
                         metric='smape', agg=agg)

##### Analysis of Residuals

In [ ]:
_ = plot_residual_diagnostics(
    winter_F1_day_df,
    models=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    id='F1',
    highlight_dayofweek=True,
    alpha=1.0,
    base_height_per_panel=250,
    exog_vars=['temperature'],
    aux_df=aux_df,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F1_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F1_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    id='F1',
    highlight_dayofweek=False,
    order_of_models=['SARIMAX', 'LSTM', 'XGBoost'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F1', 'week')]
times_df = times[('F1', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F1_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F1_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)    

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
cutoffs_all = autumn_F1_week_df['cutoff'].sort_values().unique()

fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F1_week_df[autumn_F1_week_df['cutoff'].isin(cutoffs_all)],
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F1_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F1_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F1_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F1_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F1_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F1_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

## Series F2

In [ ]:
plot_target_series(heat_df, 'F2')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F2')

### Daily horizon

In [ ]:
cv_df = merged_results[('F2', 'day')]
times_df = times[('F2', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F2_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F2_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F2_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='smape',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)   

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F2_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F2_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
winter_target = heat_df[
    (heat_df['unique_id'] == 'F2') &
    (heat_df['ds'] >= pd.Timestamp('2024-12-21')) &
    (heat_df['ds'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F2_day_df)

In [ ]:
corrupted_cutoffs = [
    pd.Timestamp('2025-01-20T23:00:00'),
    pd.Timestamp('2025-01-21T23:00:00'),
    pd.Timestamp('2025-01-22T23:00:00')
]
all_results = evaluate_cv_forecasts(
    cv_df=winter_F2_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20')),
)
summary = cv_evaluation_summary(
    all_results,
    ignore_cutoffs=corrupted_cutoffs
)

_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost',
)
ld_summary = cv_evaluation_summary(
    all_loss_diffs,
    ignore_cutoffs=corrupted_cutoffs
)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

##### Analysis of per-hour-ahead errors

In the winter period, when forecasting accuracy is most critical due to peak heat demand, the series exhibits lower heteroskedasticity because the regime transitions typical of autumn and spring do not occur. Therefore, we include an additional analysis of forecast errors by hour ahead horizon, which, given that forecasts always start at midnight, can also be interpreted as an analysis by hour of day.

In [ ]:
by_horizon_dict = by_horizon_preds(winter_F2_day_df)
per_h_summ = compute_error_stats_by_horizon(
    by_horizon_dict=by_horizon_dict,
    target_df=heat_df,
    nmae_period=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21')),
    step=1,
    log_every=5,
    show_per_cutoff=True,
    evaluate_fn=evaluate_cv_forecasts,
    summarize_fn=partial(
        cv_evaluation_summary,
        ignore_cutoffs=corrupted_cutoffs
    ),
    by_cutoff_plot_fn=partial(
        plotly_cv_metric_by_cutoff, 
        models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR'],
        width=2000
    ),
)


In [ ]:
# Target grouped by month-dayhour
plotly_daily_seasonality(
    winter_target,
    show_legend=True
)

# Errors gouped by horizon
agg = 'mean'
plotly_models_vs_horizon(per_h_summ=per_h_summ, metric='mae', agg=agg)
plotly_models_vs_horizon(per_h_summ=per_h_summ, metric='smape', agg=agg)

##### Analysis of Residuals

In [ ]:
data = plot_residual_diagnostics(
    winter_F2_day_df[~winter_F2_day_df['cutoff'].isin(corrupted_cutoffs)],
    models=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F2_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F2_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F2_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F2_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)    

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F2_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F2', 'week')]
times_df = times[('F2', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F2_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F2_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F2_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F2_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F2_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F2_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F2_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F2_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F2_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F2_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df)  

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F2_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

## Series F3

In [ ]:
plot_target_series(heat_df, 'F3')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F3')

### Daily horizon

In [ ]:
cv_df = merged_results[('F3', 'day')]
times_df = times[('F3', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F3_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F3_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F3_day_df,#[['XGBoost', 'SVMR', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'SVMR'],
    width=2000,
)
display_scrollable(fig)   

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F3_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F3_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F3_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F3_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F3_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F3_day_df,#[['LSTM', 'SVMR', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SVMR'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'SVMR'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F3', 'week')]
times_df = times[('F3', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F3_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F3_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F3_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F3_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F3_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F3_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F3_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F3_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F3_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F3_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F3_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df)  

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'MSTL', 'SVMR'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-MSTL', 'LD-SVMR'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F3_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'MSTL', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

## Series F4

In [ ]:
plot_target_series(heat_df, 'F4')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F4')

### Daily horizon

In [ ]:
cv_df = merged_results[('F4', 'day')]
times_df = times[('F4', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F4_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F4_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F4_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F4_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F4_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F4_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F4_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F4_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F4_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F4_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F4_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SVMR'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SVMR', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F4_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F4', 'week')]
times_df = times[('F4', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F4_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F4_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F4_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F4_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F4_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F4_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F4_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SARIMAX'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics 
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F4_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F4_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F4_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F4_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F4_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

## Series F5

In [ ]:
plot_target_series(heat_df, 'F5')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F5')

### Daily horizon

In [ ]:
cv_df = merged_results[('F5', 'day')]
times_df = times[('F5', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F5_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F5_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F5_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F5_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR','SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F5_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F5_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F5_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F5_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F5_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F5_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F5_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SVMR'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SVMR', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F5_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F5', 'week')]
times_df = times[('F5', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F5_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] <= pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F5_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F5_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F5_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F5_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F5_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F5_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F5_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    order_of_models=['SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F5_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F5_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F5_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F5_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

## Thesis Figures

In [ ]:
from heat_forecast.utils.plotting import extract_subplot
def set_plotly_style(fig):
    fig.update_layout(
        font=dict(size=12),
        title_font=dict(size=14),
        legend=dict(font=dict(size=12)),
    )
    fig.update_xaxes(tickfont=dict(size=12), title_font=dict(size=12))
    fig.update_yaxes(tickfont=dict(size=12), title_font=dict(size=12))
    return fig
def gray_shade(intensity: float):
    c = np.clip(1-intensity, 0, 1)
    return f"rgb({int(c*255)},{int(c*255)},{int(c*255)})"

In [ ]:
cutoffs = winter_F1_day_df['cutoff'].unique()
selected_cutoffs = cutoffs[(cutoffs > pd.Timestamp('2025-02-15 00:00:00')) & (cutoffs < pd.Timestamp('2025-03-02 00:00:00'))]
winter_F1_day_df_selected = winter_F1_day_df[winter_F1_day_df['cutoff'].isin(selected_cutoffs)]

fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F1_day_df_selected,
    start_offset=0,
    end_offset=0,
    models=['MSTL'],
    id='F1',
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=200,
    width_per_day=45,
    exog_vars=['temperature'],
    aux_df=aux_df,
    grayscale_safe=True,
    title=None
)
fig = set_plotly_style(fig)
for r in range(1, 3):
    fig.update_xaxes(showgrid=True, tickformat=r"%b %d", row=r, col=1)
print(f"Figure width: {fig.layout.width} px")
print(f"Figure height: {fig.layout.height} px")
# import plotly.io as pio
# pio.write_html(
#     fig, "fig.html",
#     include_plotlyjs="inline", full_html=True,
#     config={"toImageButtonOptions": {"format": "svg"}}
# )
import plotly.io as pio
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LSTM', 'XGBoost'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=True,
    width=650
)
fig.data[1].line.color = gray_shade(0.4)
fig.update_layout(
    legend=dict(
        x=0.8,          
        y=1.0,          
        bgcolor="rgba(255,255,255,1)",  # legend background
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    title_text=None,
    margin=dict(t=20),
)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = ""
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LSTM', 'XGBoost', 'SARIMAX'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=True,
    width=650
)
fig.data = fig.data[:1] + fig.data[2:]
fig.data[1].line.color = gray_shade(0.4)
fig.update_layout(
    legend=dict(
        x=0.8,          
        y=1.0,          
        bgcolor="rgba(255,255,255,1)",  # legend background
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    title_text=None,
    margin=dict(t=20),
)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = ""
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LSTM', 'XGBoost'],
    aux_df=aux_df,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=True,
    width=650
)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = ""
fig = extract_subplot(fig, 2, 1)
fig.update_layout(
    height=300,
    width=650,
    margin=dict(l=60, r=20, t=20, b=60),
    showlegend=False
)
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig


In [ ]:
all_maes = all_results[all_results['metric'] == 'mae']
corr_LSTM_XGB = pd.Series.corr(all_maes['LSTM'], all_maes['XGBoost'])
print("Correlation between LSTM and XGBoost MAE errors:", corr_LSTM_XGB)
corr_LSTM_SARIMAX = pd.Series.corr(all_maes['LSTM'], all_maes['SARIMAX'])
print("Correlation between LSTM and SARIMAX MAE errors:", corr_LSTM_SARIMAX)
corr_SARIMAX_XGB = pd.Series.corr(all_maes['SARIMAX'], all_maes['XGBoost'])
print("Correlation between SARIMAX and XGBoost MAE errors:", corr_SARIMAX_XGB)

In [ ]:
by_horizon_dict = by_horizon_preds(winter_F1_day_df)
per_h_summ = compute_error_stats_by_horizon(
    by_horizon_dict=by_horizon_dict,
    target_df=heat_df,
    nmae_period=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21')),
    log_every=None,
    show_per_cutoff=True,
    evaluate_fn=evaluate_cv_forecasts,
    summarize_fn=cv_evaluation_summary,
    by_cutoff_plot_fn=partial(
        plotly_cv_metric_by_cutoff, 
        models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR'],
        width=2000
    ),
)

In [ ]:
# Target grouped by month-dayhour
fig1 = plotly_daily_seasonality(
    winter_target,
    show_legend=True,
    colors=px.colors.qualitative.Dark2,
    display_fig=False,
    width=650,
    height_per_id=300
)
for i in range(len(fig1.layout.annotations)):
    fig1.layout.annotations[i].text = ""
fig1.update_layout(
    margin=dict(l=60, r=20, t=20, b=60),
    title_text=None,
)
pio.write_html(
    fig1, "fig1.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
display(fig1)
# Errors gouped by horizon
agg = 'mean'
fig2 = plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                         models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR', 'MSTL', 'Naive24h'],
                         metric='mae', 
                         agg=agg,
                         display_fig=False,
                         width=650, height=320)
for i in range(len(fig2.layout.annotations)):
    fig2.layout.annotations[i].text = ""
fig2.update_layout(
    margin=dict(l=60, r=20, t=20, b=60),
    title_text=None,
)
pio.write_html(
    fig2, "fig2.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
display(fig2)
fig3 = plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                         models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR', 'MSTL', 'Naive24h'],
                         metric='smape', agg=agg,
                         display_fig=False,
                         width=650, height=320)
for i in range(len(fig3.layout.annotations)):
    fig3.layout.annotations[i].text = ""
fig3.update_layout(
    margin=dict(l=60, r=20, t=20, b=60),
    title_text=None,
)
pio.write_html(
    fig3, "fig3.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
display(fig3)